# **Importacion de Librerias**

In [23]:
import pandas as pd
import sqlite3
from sqlalchemy import create_engine

# **Lectura de CSV**

In [24]:
con = create_engine("sqlite://", echo=False)
df = pd.read_csv('games.csv')
df.to_sql("vg", con, if_exists="replace")
df

,name,platform,release_date,summary,metascore,userscore
0,! SPACE ACCIDENT !,PC,"February 10, 2022",The year is 2119. A turning point has begun in...,tbd,tbd
1,! That Bastard Is Trying To Steal Our Gold !,PC,"May 11, 2014",It's a fun puzzle game where you need to steal...,tbd,3.4
2,!4RC4N01D!,PC,"January 12, 2018",Hardcore arkanoid in the spirit of old games f...,tbd,4.0
3,!4RC4N01D! 2: Retro Edition,PC,"February 6, 2018","Everyone dreams of returning 2007, but no one ...",tbd,3.8
4,!4RC4N01D! 3: Cold Space,PC,"March 8, 2018",!4RC4N01D! returns! This time we have an accid...,tbd,3.0
...,...,...,...,...,...,...
142412,{Undefined},PC,"August 20, 2021","A sandbox survival game, set on life-sized vox...",tbd,tbd
142413,~ Daydream ~,PC,"May 26, 2022",~Daydream~is an action 2D platform jumping gam...,tbd,tbd
142414,~Azur Ring~virgin and slave's phylacteries,PC,"July 17, 2020",New DIABLO-Like adventure game. Legendary gear...,tbd,tbd
142415,~Gigantify~,PC,"January 23, 2022",This is a small Puzzle Platformer where you dy...,tbd,tbd


# **Limpieza de Datos**

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 142417 entries, 0 to 142416
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   name          142417 non-null  object
 1   platform      142417 non-null  object
 2   release_date  142417 non-null  object
 3   summary       124702 non-null  object
 4   metascore     142417 non-null  object
 5   userscore     142417 non-null  object
dtypes: object(6)
memory usage: 6.5+ MB


ve si hay nulos

In [26]:
df.isnull().sum()

name                0
platform            0
release_date        0
summary         17715
metascore           0
userscore           0
dtype: int64

como son pocos se eliminan los nulos

In [27]:
df.dropna(inplace=True)

cambia tipo de variable a date

In [28]:
df['release_date'] = pd.to_datetime(df.release_date)
df['release_date'] = df['release_date'].dt.strftime('%Y-%m-%d')

se elimina el espacio para la columna platform

In [29]:
df['platform'] = df['platform'].str.lstrip()
df['platform'].unique()

array(['PC', 'Wii', 'iOS', 'PlayStation 4', 'Switch', 'Xbox One',
       'PlayStation Vita', 'PlayStation 3', 'PlayStation 5', 'Xbox 360',
       'PlayStation 2', 'Wii U', 'PlayStation', 'DS', 'Nintendo 64',
       '3DS', 'PSP', 'GameCube', 'Dreamcast', 'Xbox', 'Xbox Series X',
       'Stadia', 'Game Boy Advance'], dtype=object)

In [30]:
df.to_sql("vg", con, if_exists="replace")

124702

# **Analisis de Datos**

¿Cuáles son los 10 juegos más populares?

por metascore

In [32]:
consulta = '''SELECT name, summary, AVG(metascore) meta_score_avg
              FROM vg
              GROUP BY 1
              ORDER BY 3 DESC
              LIMIT 10'''
data = pd.read_sql_query(consulta, con)
data

,name,summary,meta_score_avg
0,Super Mario Odyssey,New Evolution of Mario Sandbox-Style Gameplay....,97.0
1,Super Mario Galaxy 2,"Super Mario Galaxy 2, the sequel to the galaxy...",97.0
2,Super Mario Galaxy,[Metacritic's 2007 Wii Game of the Year] The u...,97.0
3,NFL 2K1,"In the end, NFL 2K1 is a deeper, more refined ...",97.0
4,Metroid Prime,Samus returns in a new mission to unravel the ...,97.0
5,The Legend of Zelda: Breath of the Wild,Ignore everything you know about The Legend of...,96.5
6,World of Goo HD,World of Goo is a multiple award winning physi...,96.0
7,Uncharted 2: Among Thieves,Fortune hunter Nathan Drake returns in Unchart...,96.0
8,The Legend of Zelda: The Wind Waker,In this ninth entry in the Legend of Zelda ser...,96.0
9,The House in Fata Morgana - Dreams of the Reve...,A gothic suspense tale set in a cursed mansion...,96.0


por review de usuarios

In [34]:
consulta = '''SELECT name, summary, ROUND(AVG(userscore),1) user_review_avg
              FROM vg
              WHERE userscore != 'tbd'
              GROUP BY 1
              ORDER BY 3 DESC
              LIMIT 10'''
data = pd.read_sql_query(consulta, con)
data

,name,summary,user_review_avg
0,Monster Hunter Rise / Monster Hunter Rise: Sun...,"Experience the entire storyline, from the Ramp...",10.0
1,Might & Magic Elemental Guardians,Enter the world of Might & Magic: Elemental Gu...,9.8
2,Mega Man Mobile,It's MEGA MAN versus the powerful leaders and ...,9.8
3,LifeAfter,A blizzard of devastating virus beleaguered th...,9.8
4,Invizimals: Battle of the Hunters,Invizimals are invisible mythical creatures ma...,9.8
5,Arcade Archives: Bubble Bobble,THESE ARE TWO HUNGRY DINO-MIGHTS AND THEY'VE G...,9.8
6,Z.H.P. Unlosing Ranger vs Darkdeath Evilman,"Known as ZettaiHero Keikakuin Japan, Z.H.P. is...",9.7
7,Sneaky Sasquatch,"Live the life of a Sasquatch and do regular, e...",9.7
8,Resident Evil (2002),Raccoon City has been completely overrun by mu...,9.7
9,OCO,Immerse yourself in the minimalist audio-visua...,9.7


¿Cuáles son los 10 juegos menos populares?

por metascore

In [35]:
consulta = '''SELECT name, summary, AVG(metascore) meta_score_avg
              FROM vg
              GROUP BY 1
              ORDER BY 3
              LIMIT 10'''
data = pd.read_sql_query(consulta, con)
data

,name,summary,meta_score_avg
0,! SPACE ACCIDENT !,The year is 2119. A turning point has begun in...,0.0
1,! That Bastard Is Trying To Steal Our Gold !,It's a fun puzzle game where you need to steal...,0.0
2,!4RC4N01D!,Hardcore arkanoid in the spirit of old games f...,0.0
3,!4RC4N01D! 2: Retro Edition,"Everyone dreams of returning 2007, but no one ...",0.0
4,!4RC4N01D! 3: Cold Space,!4RC4N01D! returns! This time we have an accid...,0.0
5,!4RC4N01D! 4: KOHBEEP edition,The legendary Arkanoid of your childhood is ba...,0.0
6,!Anyway!,Do you like parkour? Dashing retro times? AnyW...,0.0
7,!Arre Unicornio!,Arre Unicornio! is an online multiplayer video...,0.0
8,!BurnToDie!,"!BurnToDie! - Hardcore platformer, in which yo...",0.0
9,!Dead Pixels Adventure!,Dead Pixels Adventure! - A unique retro game w...,0.0


por review de usuarios

In [36]:
consulta = '''SELECT name, summary, ROUND(AVG(userscore),1) user_review_avg
              FROM vg
              WHERE userscore != 'tbd'
              GROUP BY 1
              ORDER BY 3
              LIMIT 10'''
data = pd.read_sql_query(consulta, con)
data

,name,summary,user_review_avg
0,Kingdom Hearts HD 2.8 Final Chapter Prologue C...,Kingdom Hearts HD 2.8 Final Chapter Prologue i...,0.0
1,Pooplers,The game is diversified by power-ups and a mot...,0.0
2,Table Tennis,The desk sets the stage for this easy to enjoy...,0.0
3,The Bullet: Time of Revenge,Become a Bullet - eliminate bad people using m...,0.0
4,Fallen EP-1,The first 2.5D Survival Horror for your iPhone...,0.2
5,Paws & Claws Pet Resort,You love animals and they love you! Care for a...,0.2
6,Little League World Series Baseball 2022,Little League World Series Baseball 2022 captu...,0.3
7,Misk Schools Quest,"In the year 3020, earth became so polluted tha...",0.3
8,Wordify,"Wordify goal - using the given letters, combin...",0.3
9,Diablo Immortal,Diablo® Immortal is a mobile Massively Multipl...,0.4


¿Cuál es el mejor juego para cada plataforma?

por metascore

In [13]:
consulta = '''SELECT platform, name, summary, MAX(metascore) meta_score_max
              FROM vg
              GROUP BY 1'''
data = pd.read_sql_query(consulta, con)
data

,platform,name,summary,meta_score_max
0,3DS,The Legend of Zelda: Ocarina of Time 3D,The Legend of Zelda: Ocarina of Time 3D takes ...,94
1,DS,Grand Theft Auto: Chinatown Wars,Grand Theft Auto: Chinatown Wars is an entirel...,93
2,Dreamcast,SoulCalibur,"This is a tale of souls and swords, transcendi...",98
3,Game Boy Advance,The Legend of Zelda: A Link to the Past,Now you have a link to one of the greatest adv...,95
4,GameCube,Metroid Prime,Samus returns in a new mission to unravel the ...,97
5,Nintendo 64,The Legend of Zelda: Ocarina of Time,"As a young boy, Link is tricked by Ganondorf, ...",99
6,PC,Disco Elysium: The Final Cut,Disco Elysium - The Final Cut is the definitiv...,97
7,PSP,God of War: Chains of Olympus,[Metacritic's 2008 PSP Game of the Year] Unlea...,91
8,PlayStation,Tony Hawk's Pro Skater 2,As most major publishers' development efforts ...,98
9,PlayStation 2,Tony Hawk's Pro Skater 3,Challenge up to four friends in online competi...,97


por review de usuarios

In [37]:
consulta = '''SELECT platform, name, summary, MAX(userscore) user_review_avg
              FROM vg
              WHERE userscore != 'tbd'
              GROUP BY 1'''
data = pd.read_sql_query(consulta, con)
data

,platform,name,summary,user_review_avg
0,3DS,Fire Emblem Warriors: Fates Pack,This DLC is themed around Fire Emblem Fates.\r...,9.7
1,DS,Ghost Trick: Phantom Detective,Ghost Trick is a story of mystery and intrigue...,9.7
2,Dreamcast,Resident Evil Code: Veronica,Resident Evil Code: Veronica takes up the stor...,9.5
3,Game Boy Advance,Car Battler Joe,Rampaging rogues roam the highways and byways ...,9.5
4,GameCube,Resident Evil (2002),Raccoon City has been completely overrun by mu...,9.7
5,Nintendo 64,Banjo-Kazooie,Trouble is brewing on Spiral Mountain! Gruntil...,9.2
6,PC,Resident Evil,Raccoon City has been completely overrun by mu...,9.8
7,PSP,Z.H.P. Unlosing Ranger vs Darkdeath Evilman,"Known as ZettaiHero Keikakuin Japan, Z.H.P. is...",9.7
8,PlayStation,Resident Evil 3: Nemesis,A month and a half have passed since the mansi...,9.6
9,PlayStation 2,Resident Evil 4,In Resident Evil 4 players are reacquainted wi...,9.3


¿En qué años se lanzaron los juegos más populares?

por metascore

In [41]:
consulta = '''SELECT strftime('%Y', release_date) release_year, ROUND(AVG(metascore),1) meta_score_avg, COUNT(metascore) meta_score_count
              FROM vg
              GROUP BY 1
              ORDER BY 2 DESC'''
data = pd.read_sql_query(consulta, con)
data

,release_year,meta_score_avg,meta_score_count
0,2002,56.4,1111
1,2001,54.0,803
2,2003,53.4,1182
3,2004,49.9,1117
4,2000,48.8,608
5,2005,48.1,1374
6,2006,41.0,1631
7,2007,35.3,1977
8,2008,32.8,2217
9,2009,30.2,2821


por review de usuarios

In [39]:
consulta = '''SELECT strftime('%Y', release_date) release_year, ROUND(AVG(userscore),2) user_review_avg, COUNT(userscore) user_review_count
              FROM vg
              WHERE userscore != 'tbd'
              GROUP BY 1
              ORDER BY 2 DESC'''
data = pd.read_sql_query(consulta, con)
data

,release_year,user_review_avg,user_review_count
0,1999,8.14,163
1,1997,7.94,136
2,1998,7.89,145
3,1993,7.47,7
4,2001,7.45,580
5,2003,7.42,912
6,1991,7.42,5
7,2000,7.40,417
8,1990,7.38,4
9,2004,7.36,827


¿Qué plataformas tenían los juegos más populares?

por metascore

In [42]:
consulta = '''SELECT platform plataforma, ROUND(AVG(metascore),1) meta_score_avg, COUNT(metascore) meta_score_count
              FROM vg
              GROUP BY 1
              ORDER BY 2 DESC'''
data = pd.read_sql_query(consulta, con)
data

,plataforma,meta_score_avg,meta_score_count
0,Dreamcast,71.8,144
1,Nintendo 64,71.1,94
2,Xbox,67.3,861
3,GameCube,64.9,545
4,PlayStation,63.1,277
5,PlayStation 2,57.4,1816
6,Game Boy Advance,51.8,750
7,PSP,44.3,922
8,Xbox 360,43.0,2885
9,PlayStation 3,33.8,3049


por review de usuarios

In [43]:
consulta = '''SELECT platform plataforma, ROUND(AVG(userscore),2) user_review_avg, COUNT(userscore) user_review_count
              FROM vg
              WHERE userscore != 'tbd'
              GROUP BY 1
              ORDER BY 2 DESC'''
data = pd.read_sql_query(consulta, con)
data

,plataforma,user_review_avg,user_review_count
0,Dreamcast,7.81,138
1,Nintendo 64,7.61,94
2,PlayStation,7.59,224
3,PlayStation 2,7.52,1550
4,GameCube,7.46,483
5,Game Boy Advance,7.38,505
6,Xbox,7.21,709
7,PSP,7.07,695
8,Stadia,7.05,219
9,PlayStation Vita,7.04,612


¿Que tiene mas popularidad las consolas domesticas, portatiles, hibridas o el pc?

por metascore

In [44]:
consulta = '''SELECT CASE WHEN platform in ('3DS', 'DS', 'Game Boy Advance', 'PSP', 'PlayStation Vita') then 'portatiles'
                          WHEN platform = 'Switch' THEN 'Hibrida'
                          WHEN platform = 'PC' THEN 'PC' ELSE 'Domestica'
                          END plataforma,
                     ROUND(AVG(metascore),1) meta_score_avg, COUNT(metascore) meta_score_count
              FROM vg
              GROUP BY 1
              ORDER BY 2 DESC'''
data = pd.read_sql_query(consulta, con)
data

,plataforma,meta_score_avg,meta_score_count
0,portatiles,32.4,6397
1,Domestica,29.2,40418
2,Hibrida,20.5,9221
3,PC,7.5,68666


por review de usuarios

In [45]:
consulta = '''SELECT CASE WHEN platform in ('3DS', 'DS', 'Game Boy Advance', 'PSP', 'PlayStation Vita') then 'portatiles'
                          WHEN platform = 'Switch' THEN 'Hibrida'
                          WHEN platform = 'PC' THEN 'PC' ELSE 'Domestica'
                          END plataforma,
                      ROUND(AVG(userscore),2) user_review_avg, COUNT(userscore) user_review_count
              FROM vg
              WHERE userscore != 'tbd'
              GROUP BY 1
              ORDER BY 2 DESC'''
data = pd.read_sql_query(consulta, con)
data

,plataforma,user_review_avg,user_review_count
0,portatiles,7.03,3211
1,Hibrida,6.86,2570
2,PC,6.76,12334
3,Domestica,6.70,17149
